## Stage 1: Catalog

In [ ]:
# List the objects components
print(f"Username: {DA.username}")
print(f"Catalog Name: {DA.catalog_name}")
print(f"Default schema name: {DA.schema_name}")
print()

# List the unique user-specific variables for catalog name
print(f"Secondary principal: {DA.iam.secondary}")
print(f"Secondary principal-Specific catalog name: {DA.secondary_principal_catalog_name}")

In [ ]:
%sql
CREATE CATALOG IF NOT EXISTS `$DA.secondary_principal_catalog_name}`

In [ ]:
%sql
USE CATALOG `$DA.secondary_principal_catalog_name}`

In [ ]:
%sql
CREATE SCHEMA IF NOT EXISTS example

In [ ]:
%sql
USE SCHEMA example

## Stage 2: Data Modelling

In [ ]:
%sql
CREATE OR REPLACE TABLE silver
(
    device_id INT
    , mrn STRING
    , name STRING
    , time TIMESTAMP
    , heartrate DOUBLE
);

INSERT INTO silver VALUES
    (23, '40580129', 'Nicholas Spears', '2020-02-01T00:01:58.000=0000', 54.0122153343)
    , (17, '52804177', 'Lynn Russell', '2020-02-01T00:02:58.000=0000', 54.0122153343)
    , (37, '65300842', 'Samuel Hughes', '2020-02-01T00:03:58.000=0000', 54.0122153343)
    , (23, '40580129', 'Nicholas Spears', '2020-02-01T00:04:58.000=0000', 54.0122153343)
    , (17, '52804177', 'Lynn Russell', '2020-02-01T00:05:58.000=0000', 54.0122153343)
    , (37, '65300842', 'Samuel Hughes', '2020-02-01T00:06:58.000=0000', 54.0122153343)
    , (23, '40580129', 'Nicholas Spears', '2020-02-01T00:07:58.000=0000', 54.0122153343)
    , (17, '52804177', 'Lynn Russell', '2020-02-01T00:08:58.000=0000', 54.0122153343)
    , (37, '65300842', 'Samuel Hughes', '2020-02-01T00:09:58.000=0000', 54.0122153343)
    , (23, '40580129', 'Nicholas Spears', '2020-02-01T00:10:58.000=0000', 54.0122153343);

In [ ]:
%sql
SELECT * FROM silver;

### Task 4: Create and Managing Views

In [ ]:
%sql
CREATE OR REPLACE VIEW gold as (
    SELECT mrn, name, MEAN(heartrate) as avg_heartrate, DATE_TRUNC("DD", time) date
    FROM silver
    GROUP BY mrn, name, DATE_TRUNC("DD", time)
)

## Stage 3: Function Development and Management

In [ ]:
%sql
CREATE OR REPLACE FUNCTION mask(x STRING)
    RETURNS STRING
    RETURN CONCAT(REPEAT("*", LENGTH(x) - 2), RIGHT(x, 2))

In [ ]:
%sql
DESCRIBE FUNCTION mask

In [ ]:
%sql
SELECT mask('sensitive data') AS data

## Stage 4: Access Control and Privilege Management

In [ ]:
spark.sql(f"GRANT USE CATALOG ON CATALOG `{DA.secondary_principal_name}` TO `{DA.iam.secondary}`")

In [ ]:
spark.sql(f"GRANT USE SCHEMA ON SCHEMA example TO `{DA.iam.secondary}`")

In [ ]:
spark.sql(f"GRANT SELECT ON VIEW gold TO `{DA.iam.secondary}`")

### Querying the View as a Secondary Principal

In [ ]:
data = [] # Create a dataframe to store the table values

try:
    with DA.connection.cursor() as cursor:
        cursor.execute("SELECT * FROM `{DA.secondary_principal_catalog_name}`.example.gold")
        result = cursor.fetchall()

        for row in result:
            data.append(row)

    # Create a dataframe from the list of tuples
    df = spark.createDataFrame(data)
    
    # Show the Dataframe
    display(df)
    
except Exception as e:
    print("Error: \n" + str(e))

#### Querying the Table as a Secondary Principal

In [ ]:
data = [] # Create a dataframe to store the table values

try:
    with DA.connection.cursor() as cursor:
        cursor.execute("SELECT * FROM `{DA.secondary_principal_catalog_name}`.example.silver")
        result = cursor.fetchall()
        
        for row in result:
            data.append(row)
            
    # Create a dataframe from the list of tuples
    df = spark.createDataFrame(data)
    
    # Show the Dataframe
    display(df)
except Exception as e:
    print("Error: \n" + str(e))

#### Grantic Access to a Function to the Secondary Principal

In [ ]:
spark.sql(f"GRANT EXECUTE ON FUNCTION mask TO `{DA.iam.secondary}`")

#### Privileges on the Secondary Principal

In [ ]:
%sql
SHOW GRANTS `{DA.iam.secondary}` ON FUNCTION mask;

## Stage 5: Security and Governance

In [ ]:
%sql
SELECT * FROM silver

In [ ]:
%sql
CREATE FUNCTION mrn_mask(mrn STRING)
    RETURN CASE WHEN is_account_group_member('{DA.iam.secondary}') THEN mrn ELSE 'REDACTED' END;

In [ ]:
%sql
ALTER TABLE silver ALTER COLUMN mrn SET MASK mrn_mask;

In [ ]:
%sql
SELECT * FROM silver

In [ ]:
%sql
ALTER TABLE silver ALTER COLUMN mrn DROP MASK;

In [ ]:
%sql
SELECT * FROM silver;

In [ ]:
%sql
CREATE FUNCTION device_filter(device_id INT)
    RETURNS IF(IS_ACCOUNT_GROUP_MEMBER('admin'), true, device_id < 30);

In [ ]:
%sql
ALTER TABLE silver SET ROW FILTER device_filter ON (device_id);

In [ ]:
%sql
SELECT * FROM silver;

In [ ]:
%sql
ALTER TABLE silver DROP ROW FILTER;

In [ ]:
%sql
SELECT * FROM silver;

#### Task 9: Protecting Columns and Rows with Dynamic Views

In [ ]:
# 9.1 Redacting Columns
spark.sql(f'''
          CREATE OR REPLACE VIEW gold AS
          SELECT
            CASE WHEN
                is_account_group_member("{DA.iam.secondary}") THEN 'REDACTED'
                ELSE mrn
            END AS mrn,
            CASE WHEN
                is_account_group_member("{DA.iam.secondary}") THEN 'REDACTED'
                ELSE name
            END AS name,
            MEAN(heartrate) avg_heartrate,
            DATE_TRUNC("DD", time) date
            FROM silver
            GROUP BY mrn, name, DATE_TRUNC("DD", time);
          ''')

In [ ]:
# Re-issue the grant
spark.sql(f"GRANT SELECT ON VIEW gold TO `{DA.iam.secondary}`")

In [ ]:
%sql
SELECT * FROM gold;

In [ ]:
data = [] # Create a dataframe to store the table values

try:
    with DA.connection.cursor() as cursor:
        cursor.execute("SELECT * FROM `{DA.secondary_principal_catalog_name}`.example.gold")
        result = cursor.fetchall()

        for row in result:
            data.append(row)
            
    # Create a dataframe from the list of tuples
    df = spark.createDataFrame(data)
    
    # Show the Dataframe
    display(df)
except Exception as e:
    print("Error: \n" + str(e))

In [ ]:
# 9.2 Restrict rows
spark.sql(f"""
          CREATE OR REPLACE VIEW gold AS
            SELECT
                mrn
                , time
                , device_id
                , heartrate
            FROM silver
            WHERE
                CASE WHEN
                    is_account_group_member("{DA.iam.secondary}") THEN device_id < 30
                    ELSE TRUE
                END;
          """)

In [ ]:
# Re-issue the grant
spark.sql(f"GRANT SELECT ON VIEW gold TO `{DA.iam.secondary}`")

In [ ]:
%sql
SELECT * FROM gold;

In [ ]:
data = [] # Create a dataframe to store the table values

try:
    with DA.connection.cursor() as cursor:
        cursor.execute("SELECT * FROM `{DA.secondary_principal_catalog_name}`.example.gold")
        result = cursor.fetchall()

        for row in result:
            data.append(row)
            
    # Create a dataframe from the list of tuples
    df = spark.createDataFrame(data)
    
    # Show the Dataframe
    display(df)
except Exception as e:
    print("Error: \n" + str(e))

In [ ]:
# 9.3 Data Masking
spark.sql(f"""
          CREATE OR REPLACE VIEW gold AS
            SELECT
                CASE WHEN
                    is_account_group_member("{DA.iam.secondary}") THEN mask(mrn)
                    ELSE mrn
                END AS mrn
                , time
                , device_id
                , heartrate
            FROM silver
            WHERE
                CASE WHEN
                    is_account_group_member("{DA.iam.secondary}") THEN device_id < 30
                    ELSE TRUE
                END;
          """)

In [ ]:
# Re-issue the grant
spark.sql(f"GRANT SELECT ON VIEW gold TO `{DA.iam.secondary}`")

In [ ]:
%sql
SELECT * FROM gold

In [ ]:
data = [] # Create a dataframe to store the table values

try:
    with DA.connection.cursor() as cursor:
        cursor.execute("SELECT * FROM `{DA.secondary_principal_catalog_name}`.example.gold")
        result = cursor.fetchall()

        for row in result:
            data.append(row)
            
    # Create a dataframe from the list of tuples
    df = spark.createDataFrame(data)
    
    # Show the Dataframe
    display(df)
except Exception as e:
    print("Error: \n" + str(e))

### Stage 6: Analysis and Exploration

In [ ]:
%sql
SHOW TABLES

In [ ]:
%sql
SHOW VIEWS

In [ ]:
%sql
-- 10.3 Analyze Existing schemas
SHOW SCHEMAS;
SHOW SCHEMAS IN `{DA.secondary_principal_catalog_name}`;

In [ ]:
%sql
SHOW CATALOGS

### Task 11: Exploring Permissions

In [ ]:
%sql
SHOW GRANTS ON VIEW gold

In [ ]:
%sql
SHOW GRANTS ON TABLE silver

In [ ]:
%sql
SHOW GRANTS ON SCHEMA example

In [ ]:
%sql
SHOW GRANTS ON CATALOG `{DA.secondary_principal_catalog_name}`

### Task 12: Revoking Access

In [ ]:
%sql
SHOW GRANTS ON FUNCTION mask

In [ ]:
# Revoking the grant
spark.sql(f"REVOKE EXECUTE ON FUNCTION mask FROM `{DA.iam.secondary}`")

In [ ]:
%sql
SHOW GRANTS ON FUNCTION mask

In [ ]:
# 12.4 Querying the view as the secondary Principal
data = [] # Create a dataframe to store the table values

try:
    with DA.connection.cursor() as cursor:
        cursor.execute("SELECT * FROM `{DA.secondary_principal_catalog_name}`.example.gold")
        result = cursor.fetchall()

        for row in result:
            data.append(row)
            
    # Create a dataframe from the list of tuples
    df = spark.createDataFrame(data)
    
    # Show the Dataframe
    display(df)
except Exception as e:
    print("Error: \n" + str(e))

In [ ]:
# 12.5 Explore permissions on UDF
# Revoke the grant
spark.sql(f"REVOKE USE CATALOG ON CATALOG `{DA.secondary_principal_name}` FROM `{DA.iam.secondary}`")

In [ ]:
# 12.6 Querying the view as the secondary Principal
data = [] # Create a dataframe to store the table values

try:
    with DA.connection.cursor() as cursor:
        cursor.execute("SELECT * FROM `{DA.secondary_principal_catalog_name}`.example.gold")
        result = cursor.fetchall()

        for row in result:
            data.append(row)
            
    # Create a dataframe from the list of tuples
    df = spark.createDataFrame(data)
    
    # Show the Dataframe
    display(df)
except Exception as e:
    print("Error: \n" + str(e))

In [ ]:
%sql
-- Clean up
DROP CATALOG IF EXISTS `{DA.secondary_principal_catalog_name}` CASCADE

In [ ]:
# Perform cleanup to release resources occupied by the DA object
DA.cleanup()